In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from db import ActionDatabase
from runner import process_threads, execute_due_actions
from config import CONFIG

In [47]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_config['name']: SlackInteractor(workspace_config)
    for _, workspace_config in workspaces.items()
}
llm = ClaudeLLM()

agents = {}
for workspace_name, slack_interactor in slack_interactors.items():
    action_db = ActionDatabase(workspace_name)
    agents[workspace_name] = [
        ProjectManagerAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        SarcasticAgent(llm, action_db, slack_interactor, workspace_name=workspace_name),
        PaulGrahamAgent(llm, action_db, slack_interactor, workspace_name=workspace_name)
    ]

In [48]:
for workspace_name, slack_interactor in slack_interactors.items():
    print(f"\nFetching new messages for workspace: {workspace_name}")
    data = slack_interactor.fetch_new_user_messages()
    threads = slack_interactor.organize_threads(data)
    print(f"Found {len(threads)} threads with new user messages in {workspace_name}.")

    if not slack_interactor.is_first_run:
        results = process_threads(agents[workspace_name], threads)
        print(f"\nChecking for due actions in {workspace_name}...")
        execute_due_actions(agents[workspace_name])
    else:
        print(f"First run for {workspace_name}. Skipping thread processing and due actions.")
        slack_interactor.is_first_run = False


Fetching new messages for workspace: agentflow
Set conversations_oldest to 1722399936.894109
Loaded 446 old messages
Rate limited. Retrying in 1.11 seconds (attempt 1/10)
Found 1 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 0 threads with new user messages in agentflow.

Checking for due actions in agentflow...

Fetching new messages for workspace: legal flow
Set conversations_oldest to 1724897852.811579
Loaded 18 old messages
Found 0 new messages
Updated complete_conversations_legal flow.pkl with new messages
Found 0 threads with new user messages in legal flow.

Checking for due actions in legal flow...
